In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow==2.0.0 
import tensorflow as tf
import numpy as np
import os
import time

     |████████████████████████████████| 86.3MB 104.7MB/s 
     |████████████████████████████████| 450kB 54.3MB/s 
     |████████████████████████████████| 3.8MB 45.8MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninsta

In [0]:
"""Reading the training and test text files"""
sst_train_file = open("sst_train.txt", 'rb').read()
sst_validation_file = open("sst_dev.txt", 'rb').read()
sst_test_file = open("sst_test.txt", 'rb').read()

In [3]:
# Read, then decode for py2 compat.
text = sst_train_file.decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 976668 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

__label__4	The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
__label__5	The gorgeously elaborate continuation of `` Th


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

97 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '#' :   4,
  '$' :   5,
  '&' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'__label__4\tTh' ---- characters mapped to int ---- > [56 56 69 58 59 62 69 56 56 20  0 49 65]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

_
_
l
a
b


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"__label__4\tThe Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to mak"
'e a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .\n__label__'
"5\tThe gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a colu"
"mn of words can not adequately describe co-writer/director Peter Jackson 's expanded vision of J.R.R."
" Tolkien 's Middle-earth .\n__label__4\tSinger/composer Bryan Adams contributes a slew of songs -- a fe"


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "__label__4\tThe Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to ma"
Target data: "_label__4\tThe Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to mak"


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 56 ('_')
  expected output: 56 ('_')
Step    1
  input: 56 ('_')
  expected output: 69 ('l')
Step    2
  input: 69 ('l')
  expected output: 58 ('a')
Step    3
  input: 58 ('a')
  expected output: 59 ('b')
Step    4
  input: 59 ('b')
  expected output: 62 ('e')


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           24832     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "ng Brown snake .\n__label__1\tPersonally , I 'd rather watch them on the Animal Planet .\n__label__2\tCh"

Next Char Predictions: 
 "SB8u:*K:Q0f8bJ7çWâ#XjIR+EG$Ky/ü_58RôS&QRmLáçeSTzfç'C\t-InK1d''+ñiæmj)&uháJ_FwñLp9áFG `/j\tKB,JáHILé&ôf"


In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5749097


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
EPOCHS=20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
151/151 [==============================] - 703s 5s/step - loss: 2.5833
Epoch 2/20
151/151 [==============================] - 669s 4s/step - loss: 1.8877
Epoch 3/20
151/151 [==============================] - 666s 4s/step - loss: 1.6480
Epoch 4/20
151/151 [==============================] - 666s 4s/step - loss: 1.4814
Epoch 5/20
151/151 [==============================] - 665s 4s/step - loss: 1.3734
Epoch 6/20
151/151 [==============================] - 666s 4s/step - loss: 1.2985
Epoch 7/20
151/151 [==============================] - 665s 4s/step - loss: 1.2421
Epoch 8/20
151/151 [==============================] - 665s 4s/step - loss: 1.1959
Epoch 9/20
151/151 [==============================] - 664s 4s/step - loss: 1.1540
Epoch 10/20
151/151 [==============================] - 665s 4s/step - loss: 1.1190
Epoch 11/20
151/151 [==============================] - 671s 4s/step - loss: 1.0831
Epoch 12/20
151/151 [==============================] - 665s 4s/step - loss: 1.0499
Epoch 13/20
1

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
#model.load_weights(tf.train.Checkpoint.restore('./training_checkpoints/ckpt_1.index'))

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:

print(generate_text(model, start_string=u"__label__"))

__label__4	It says it and the body of the film 's verbal power and share one and revolutional singers .
__label__4	Nice comedies that telms the filmmaker 's hero of the small pants .
__label__1	It 's refreshing that you will probably be surprising in itself is always feel going to rich turn on squarment and insane .
__label__2	Is not scarielent , your season handidge the story in almost every frame has little to other insights to nauges that I think in the right partic ` tramed with Brounghy . '
__label__4	Watching Nijinsky 's indiffer than the impressive kndg , because extremes starring role -- The Unpersonal delicate .
__label__4	I will be cynic , spiffied , and , rejected star two dialogue historical tale .
__label__4	American life .
__label__2	This l__4	Remarkable material .
__label__2	Just old-fashioned annoy Circural Average wo less for crisis .
__label__4	As far the first film in Socio Nathans that renuined to the air that 's so like an opting horror .
__label__2	The movie has s

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./training_checkpoints/ckpt_1')
model.build(tf.TensorShape([1, None]))

print(generate_text(model, start_string=u"__label__"))

__label__4	Thoud alimant , diss pabll-bel__5	Thiss pronzine fronen inombuglets bed ticcthe Dint it fuvncer .
__label__label__3	Exe muvimesil uver the whky---uplentolls , bunt quthe Aice sivhire ablis la kect dVet andill waldonker , bo omestumeaga dod -fitimcionghe whis .
__label__2YYevill for of at , baveverens coo n't thlm char hagn per ifitha ted ts af bovely t-a moul mickengara sucrite , dicr a dite bfuth tfeselo tht -ainemy , ads ofhaly chaigh stele ss ut atlay .
__label__2	Thor kivin flos 's lak is mealed Hats mop thary wof theisirole mera too wons ounfing , tha he , ef wis 's ath Dispsta cureline Ss an tees vatct , pwithiduss a film diand it thearenss feath kes coves move and ontorovea Rome chlpersens rikvel seprieng flecs Surse hacrert o foou lem-ctitis Ane fiot teretstite , and en thise fith Vabl Efwlyes , buquidly rotalizy bomperpard His of privery it 'ss timist , moted lithelt tht ande lapang medort ut , ghr sha criovuressors .
__label__2	The likg , utt wing aveilag veeks arp

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./training_checkpoints/ckpt_10')
model.build(tf.TensorShape([1, None]))

print(generate_text(model, start_string=u"__label__"))

__label__3	Ho much .
__label__3	Se inthing seem screencel_3	This is an astrebactable popioutly , but dunning to feaved its creative year .
__label__2	Ambreading simple , mores like squanderfulgar way takeve what reveel around there and often enjoying their stadful when you dinking hour , the time ; and the esame smarlent .
__label__4	It shaup over Sago I' with herosting the fiem is and touching or edited that one sevoratoly fane .
__label__4	Rid like the kilirg on the seen wit .
__label__5	Bahant of all uts crime that stows than it , husting preary the attrath throughout on chiphos werring without the irrance movie that have been movies , giventally as a senso threar high this premise .
__label__3	A truewh and can both high-landscupe -- in feeling tone that straight -- and flated work life the movie out this you 've worth the film than whatever than yatm is viewly a deligicant with -- not moviegeanize diffience .
__label__1	Going wishy comformance and madve to pretty pensemlate tale or